In [8]:
import gensim.models.keyedvectors as word2vec
import gensim.downloader as api
import pprint

pp = pprint.PrettyPrinter()

In [2]:
def load_word2vec():
    """ Load Word2Vec Vectors
        Return:
            wv_from_bin: All 3 million embeddings, each lengh 300
    """
    # wv_from_bin = api.load("word2vec-google-news-300")
    wv_from_bin = word2vec.KeyedVectors.load_word2vec_format('dataset/GoogleNews-vectors-negative300.bin', binary=True)
    vocab = list(wv_from_bin.vocab.keys())
    print("Loaded vocab size %i" % len(vocab))
    return wv_from_bin


In [3]:
wv = load_word2vec()

Loaded vocab size 3000000


In [12]:
print(wv)
print("most sim:")
result = wv.most_similar(positive=['woman', 'king'], negative=['man'])
pp.pprint(result[:10])

print("\ndoesnt match:")
print(wv.doesnt_match("breakfast cereal dinner lunch".split()))

print("\nsim:")
print(wv.similarity('woman', 'man'))

print("\nsim by word")
result = wv.similar_by_word("cat")
pp.pprint(result[:10])

most sim:
[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.518113374710083),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

doesnt match:
cereal

sim:
0.76640123

sim by word
[('cats', 0.8099379539489746),
 ('dog', 0.7609456777572632),
 ('kitten', 0.7464985251426697),
 ('feline', 0.7326233983039856),
 ('beagle', 0.7150583267211914),
 ('puppy', 0.7075453996658325),
 ('pup', 0.6934291124343872),
 ('pet', 0.6891531348228455),
 ('felines', 0.6755931377410889),
 ('chihuahua', 0.6709762215614319)]


In [14]:
from gensim.test.utils import datapath
similarities = wv.evaluate_word_pairs(datapath('wordsim353.tsv'))
print(similarities)

((0.6238773466616108, 1.7963237724170263e-39), SpearmanrResult(correlation=0.6589215888009288, pvalue=2.5346056459149263e-45), 0.0)
